# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
vacation_data = pd.read_csv("../output_data/weather_cities.csv")
vacation_data



,Unnamed: 0,City,Country,Cloudiness,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,Yangi Marg`ilon,UZ,49,1613014227,87,40.4272,71.7189,41.00,3.74
1,1,Jamestown,US,90,1613014173,86,42.0970,-79.2353,18.00,5.75
2,2,Puerto Ayora,EC,82,1613014228,99,-0.7393,-90.3518,75.00,1.01
3,3,Mataura,NZ,100,1613014228,88,-46.1927,168.8643,49.89,11.88
4,4,Comodoro Rivadavia,AR,0,1613014226,94,-45.8667,-67.5000,62.60,24.63
...,...,...,...,...,...,...,...,...,...,...
556,556,Dauphin,CA,20,1613014293,69,51.1494,-100.0502,-20.20,6.91
557,557,Norrtälje,SE,96,1613014264,92,59.7580,18.7050,21.00,8.05
558,558,Hue,VN,20,1613014293,68,16.4667,107.6000,71.60,5.75
559,559,Mitú,CO,100,1613014293,91,1.1983,-70.1733,73.53,2.57


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#configure gmaps
gmaps.configure(api_key=g_key)

# Use the Lat and Lng as locations and Humidity as the weight.
locations = vacation_data[["Lat", "Lng"]]
humidity = vacation_data["Humidity"]

In [4]:
#Add Heatmap layer to map

heatfig = gmaps.figure(center=(50.0, -5.0), zoom_level=3)
max_intensity = np.max(humidity)

heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=100, point_radius=3)


heatfig.add_layer(heat_layer)

heatfig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Narrow down the cities - wind speed less than 10 mph, cloudiness equals to 0 and max temp between 70 and 80
narrowed_cities = vacation_data.loc[(vacation_data["Wind Speed"] <= 10) & (vacation_data["Cloudiness"] == 0) & \
                                   (vacation_data["Max Temp"] >= 70) & (vacation_data["Max Temp"] <= 80)].dropna()

narrowed_cities

,Unnamed: 0,City,Country,Cloudiness,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
135,135,Imphal,IN,0,1613014252,32,24.8167,93.9500,71.47,1.81
175,175,Pisco,PE,0,1613014258,78,-13.7000,-76.2167,73.40,5.75
316,316,Bocaiúva,BR,0,1613014274,83,-17.1078,-43.8150,71.60,4.52
323,323,Sittwe,MM,0,1613014275,36,20.1500,92.9000,77.23,1.92
404,404,Salaga,GH,0,1613014281,66,8.5508,-0.5188,77.13,4.38
442,442,Conakry,GN,0,1613014011,77,9.5380,-13.6773,77.81,7.31
508,508,Cartagena,CO,0,1613014289,83,10.3997,-75.5144,78.80,9.22
534,534,Oussouye,SN,0,1613014291,83,12.4850,-16.5469,71.60,4.61


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# Store into variable named hotel_df
hotel_df = narrowed_cities.loc[:,["City","Country", "Lat", "Lng"]]

# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""

hotel_df

,City,Country,Lat,Lng,Hotel Name
135,Imphal,IN,24.8167,93.9500,
175,Pisco,PE,-13.7000,-76.2167,
316,Bocaiúva,BR,-17.1078,-43.8150,
323,Sittwe,MM,20.1500,92.9000,
404,Salaga,GH,8.5508,-0.5188,
442,Conakry,GN,9.5380,-13.6773,
508,Cartagena,CO,10.3997,-75.5144,
534,Oussouye,SN,12.4850,-16.5469,


In [15]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
   
    params["location"] = f"{lat},{lng}"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    # save the hotel name to dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        


Retrieving Results for Index 135: Imphal.
Closest hotel in Imphal is The Castle Hotel.
Retrieving Results for Index 175: Pisco.
Closest hotel in Pisco is HOSTAL MIRAMAR.
Retrieving Results for Index 316: Bocaiúva.
Closest hotel in Bocaiúva is Pontual Hotel.
Retrieving Results for Index 323: Sittwe.
Closest hotel in Sittwe is Hotel Memory.
Retrieving Results for Index 404: Salaga.
Closest hotel in Salaga is Maridon Lodge.
Retrieving Results for Index 442: Conakry.
Closest hotel in Conakry is Noom Hotel Conakry.
Retrieving Results for Index 508: Cartagena.
Closest hotel in Cartagena is Bastión Luxury Hotel.
Retrieving Results for Index 534: Oussouye.
Closest hotel in Oussouye is Campement ALULUM.


In [16]:
hotel_df

,City,Country,Lat,Lng,Hotel Name
135,Imphal,IN,24.8167,93.9500,The Castle Hotel
175,Pisco,PE,-13.7000,-76.2167,HOSTAL MIRAMAR
316,Bocaiúva,BR,-17.1078,-43.8150,Pontual Hotel
323,Sittwe,MM,20.1500,92.9000,Hotel Memory
404,Salaga,GH,8.5508,-0.5188,Maridon Lodge
442,Conakry,GN,9.5380,-13.6773,Noom Hotel Conakry
508,Cartagena,CO,10.3997,-75.5144,Bastión Luxury Hotel
534,Oussouye,SN,12.4850,-16.5469,Campement ALULUM


In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [8]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add the layer to the map
heatfig.add_layer(markers)

# Display figure
heatfig

#zoom out to see the markers


Figure(layout=FigureLayout(height='420px'))